# 1. Подготовка золотого стандарта.  
Я использовала тегсет UDpipe, потому что он довольно полный, при этом универсальный (без большого количества подробностей) и интуитивно понятный. Однако я внесла некоторые упрощения - объединила SCONJ и CCONJ под единым тэгом CONJ; DET - под тэгом ADJ, так как далеко не все тэггеры учитывают различие в союзах.
Текст для разметки я составляла из предложений с феминитивами, окказионализмами, заимствованиями, омонимами, так как все это представляет сложность для автоматического тэггинга. В случае с первыми тремя - данные слова могут быть незнакомы словарю, а в случае с омонимами - не все тэггеры могут различать омонимию, так как для этого нужно учитывать контекст. 

In [1]:
import csv
import pandas as pd

In [159]:
with open('analysis.conllu', 'r', encoding='utf-8') as f:
  file = f.readlines()

In [160]:
for i in range(len(file)):
  file[i] = file[i].split('\t')
  file[i] = file[i][1:4:2]

In [161]:
with open('golden.tsv','w') as file_out:
  file_out.write('form\ttag\n')
with open('golden.tsv','a') as file_out:
    for line in file:
        if len(line) != 0:
          string = f'{line[0]}\t{line[1]}'
          file_out.write(string)
          file_out.write('\n')

In [162]:
df = pd.read_csv('golden.tsv', delimiter='\t')

In [150]:
df

,form,tag
0,Тогда,ADV
1,бы,AUX
2,узрел,VERB
3,легендарную,ADJ
4,щуку,NOUN
...,...,...
258,работе,NOUN
259,видя,VERB
260,прекрасные,ADJ
261,ножки,NOUN


# 2. Тэггеры.  
Я буду использовать следующие тэггеры: Natasha, Pymorhy и Spacy.   

**Spacy**

In [ ]:
!python -m spacy download ru_core_news_sm

In [118]:
import spacy

In [103]:
nlp = spacy.load("ru_core_news_sm")

In [151]:
with open('trial.txt', encoding='utf-8') as f:
  text = f.read()
analized = nlp(text)

In [152]:
with open('spacy.tsv', 'w', encoding='utf-8') as f:
  f.write(f'form\ttag\n')
  for token in analized:
    f.write(f'{token}\t{token.pos_}\n')

**Pymorphy**

In [ ]:
!pip install pymorphy2

In [60]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [153]:
text_toks = word_tokenize(text)

In [154]:
answers = []
with open('pymorphy.tsv', 'w', encoding='utf-8') as f:
  f.write('form\ttag\n')
  for i in text_toks:
    if i != '':
      f.write(f'{i}\t{morph.parse(i)[0].tag.POS}\n')

**Natasha**

In [63]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

In [64]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [155]:
with open('trial.txt', encoding='utf-8') as f:
  text = f.read()

In [156]:
doc = Doc(text)
doc

Doc(text='Тогда бы узрел легендарную щуку векующую в озорно...)

In [157]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [158]:
with open('natasha.tsv', 'w', encoding='utf-8') as f:
  f.write('form\ttag\n')
  for i in doc.tokens:
    f.write(f'{i.text}\t{i.pos}\n')

# 3. Аккуратность.

Функция *change_tag* приводит все файлы к единому формату разметки.

In [213]:
import re
def change_tag(filename):
  with open(filename) as f:
    text = f.read()
  text = re.sub('CCONJ', 'CONJ', text)
  text = re.sub('SCONJ', 'CONJ', text)
  text = re.sub('PRTS', 'VERB', text) # я приняла решение засчитывать все отглагольные вещи как глаголы: деепричастия, причастия, инфинитивы
  text = re.sub('ADVB', 'ADV', text)
  text = re.sub('ADJF', 'ADJ', text)
  text = re.sub('ADJS', 'ADJ', text)
  text = re.sub('NPRO', 'PRON', text)
  text = re.sub('PRCL', 'PART', text)
  text = re.sub('DET', 'ADJ', text)
  text = re.sub('PREP', 'ADP', text)
  text = re.sub('INFN', 'VERB', text)
  text = re.sub('GRND', 'VERB', text)
  text = re.sub('PRTF', 'VERB', text)
  text = re.sub('COMP', 'ADJ', text)
  with open(filename, 'w') as f:
    f.write(text)

In [217]:
list_files = ['golden.tsv', 'pymorphy.tsv', 'natasha.tsv', 'spacy.tsv']
for i in list_files:
  change_tag(i)

In [206]:
import numpy as np
def compare_lists(filename):
  list1 = pd.read_csv('golden.tsv', delimiter='\t')['tag'].values
  list2 = pd.read_csv(filename, delimiter='\t')['tag'].values
  counter = 0

  amount = np.array(list1) == np.array(list2)
  for i in amount:
      if i:
        counter += 1
  
  accuracy = counter/len(list1)
  return accuracy

In [218]:
print(f"Accuracy for natasha: {compare_lists('natasha.tsv')}", f"Accuracy for spacy: {compare_lists('spacy.tsv')}", 
      f"Accuracy for pymorphy: {compare_lists('pymorphy.tsv')}", sep='\n')

Accuracy for natasha: 0.8897338403041825
Accuracy for spacy: 0.935361216730038
Accuracy for pymorphy: 0.8212927756653993


Что касается омонимии, есть тэггеры, которые умеют справляться с ней, в том числе UDpipe. Поэтому неразличение омонимии тэггерами выше я считала за ошибку.

# 4. N-граммы

1. **не + прилагательное**, так как отрицательная частица *не* превращает все последующее высказывание в отрицание --> придает негативную окраску (из-за специфики текста - отзыв)
2. **хорошо/плохо + прилагательное/причастие** типа *хорошо снятый*. Это улучшит анализ, так как поможет исключить позитивную идентификацию по слову *хорошо* в случаях - *хорошо, что я отдал билеты на этот фильм друзьям, ведь фильм ужасен*. (для *плохо* ситуация аналогичная + инвентарь подобных наречий можно расширить)
3. **не + глагол**, по той же причине, что в п.1